In [4]:
#rdkit関連のimport
from rdkit import Chem
from rdkit.Chem.rdDistGeom import ETKDGv3, EmbedMolecule
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule
from rdkit.Chem import PandasTools

#psi4
import psi4

#その他のimport
import pandas as pd
import time
from timeout_decorator import timeout, TimeoutError

#分子の読み込み
suppl_train = Chem.SDMolSupplier('solubility.train.sdf')  #<-train
train_mols = [x for x in suppl_train]   #<-train

suppl_test = Chem.SDMolSupplier('solubility.test.sdf')  #<-test
test_mols = [x for x in suppl_test]   #<-test

#datasetの作成
ID_list = []
smiles_list = []
SOL_list = []
SOL_Class_list = []
for mol in train_mols:
    ID_list.append(mol.GetIntProp('ID'))
    smiles_list.append(mol.GetProp('smiles'))
    SOL_list.append(mol.GetDoubleProp('SOL'))
    SOL_Class_list.append(mol.GetProp('SOL_classification'))
for mol in test_mols:
    ID_list.append(mol.GetIntProp('ID'))
    smiles_list.append(mol.GetProp('SMILES'))
    SOL_list.append(mol.GetDoubleProp('SOL'))
    SOL_Class_list.append(mol.GetProp('SOL_classification'))
    
mol_datasets = pd.DataFrame({'ID':ID_list,'smiles':smiles_list,'SOL':SOL_list,'SOL_Class':SOL_Class_list})

#Error分子
Error_mols = pd.read_table('Sol_ErrorMolecule.txt', sep = ' ')
Error_ID = Error_mols.ID
mol_datasets = mol_datasets.drop(Error_ID)
mol_datasets = mol_datasets.sort_values('ID')

#smilesからrdKitのMolオブジェクトの構築
PandasTools.AddMoleculeColumnToFrame(frame=mol_datasets, smilesCol = 'smiles')

#読み込めない分子を削除
mol_datasets[ 'MOL'] = mol_datasets.ROMol.map(lambda x: False if x == None else True)
del_index = mol_datasets[mol_datasets.MOL == False].index
mol_datasets.drop(del_index, inplace = True)
mol_datasets = mol_datasets.reset_index(drop=True)

#データ数の取得
data_num = len(mol_datasets.index)
print('start with {} samples'.format(data_num))

mol_datasets.to_csv("AvailableSolubilityDataSets.csv")

RDKit ERROR: [16:59:23] SMILES Parse Error: syntax error while parsing: n(H)(c(c(c1cccc2)ccc3)c3)c12
[16:59:23] SMILES Parse Error: syntax error while parsing: n(H)(c(c(c1cccc2)ccc3)c3)c12
RDKit ERROR: [16:59:23] SMILES Parse Error: Failed parsing SMILES 'n(H)(c(c(c1cccc2)ccc3)c3)c12' for input: 'n(H)(c(c(c1cccc2)ccc3)c3)c12'
RDKit ERROR: [16:59:23] SMILES Parse Error: syntax error while parsing: c1ccc4c(c1)ccc5c3ccc2ccccc2c3n(H)c45
RDKit ERROR: [16:59:23] SMILES Parse Error: Failed parsing SMILES 'c1ccc4c(c1)ccc5c3ccc2ccccc2c3n(H)c45' for input: 'c1ccc4c(c1)ccc5c3ccc2ccccc2c3n(H)c45'
RDKit ERROR: [16:59:23] SMILES Parse Error: syntax error while parsing: O=C(O)CC(c(c(N1(H))ccc2)c2)=C1
RDKit ERROR: [16:59:23] SMILES Parse Error: Failed parsing SMILES 'O=C(O)CC(c(c(N1(H))ccc2)c2)=C1' for input: 'O=C(O)CC(c(c(N1(H))ccc2)c2)=C1'
[16:59:23] SMILES Parse Error: Failed parsing SMILES 'n(H)(c(c(c1cccc2)ccc3)c3)c12' for input: 'n(H)(c(c(c1cccc2)ccc3)c3)c12'
[16:59:23] SMILES Parse Error: synta

start with 1258 samples
